In [10]:
!pip install transformers datasets

In [11]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from datasets import load_dataset

In [12]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [13]:
# Load the FEVER dataset
fever = load_dataset("fever", "v1.0")

# Label mappings
label2id = {"SUPPORTS": 0, "REFUTES": 1, "NOT ENOUGH INFO": 2}
id2label = {v: k for k, v in label2id.items()}

In [14]:
# # Load tokenizer and model
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3, hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2)

# tokenizer = RobertaTokenizer.from_pretrained("bert-base-uncased")
# model = RobertaForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

In [15]:
# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# Load the model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3) 

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Preprocess data
def preprocess_data(batch):
    inputs = tokenizer(batch["claim"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = [label2id[label] for label in batch["label"]]
    return inputs

In [17]:
# train_data = fever["train"].map(preprocess_data, batched=True)
# val_data = fever["labelled_dev"].map(preprocess_data, batched=True)

# train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
# val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [18]:
from datasets import Dataset

# Function to sample equal examples for each class
def sample_equal_classes(dataset, label_column, num_samples_per_class):
    sampled_data = []
    for label in label2id.values():  # Iterate over label IDs
        class_samples = dataset.filter(lambda x: label2id[x[label_column]] == label)
        sampled_data.append(class_samples.select(range(min(num_samples_per_class, len(class_samples)))))
    # Combine all class samples
    return Dataset.from_dict({key: sum([d[key] for d in sampled_data], []) for key in dataset.column_names})

# Sample 20,000 training examples and 5,000 validation examples
train_data = sample_equal_classes(fever["train"], "label", num_samples_per_class=10000 // len(label2id))
val_data = sample_equal_classes(fever["labelled_dev"], "label", num_samples_per_class=1000 // len(label2id))

# Preprocess the data
train_data = train_data.map(preprocess_data, batched=True)
val_data = val_data.map(preprocess_data, batched=True)

# Convert to PyTorch format
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Create DataLoaders
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=32)

print(f"Training examples: {len(train_data)}, Validation examples: {len(val_data)}")

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Training examples: 9999, Validation examples: 999


In [19]:
# train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
# val_dataloader = DataLoader(val_data, batch_size=128)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [20]:
from tqdm import tqdm

for epoch in range(10):  # Number of epochs
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    # Wrap the DataLoader with TQDM for a progress bar
    train_progress = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")

    for batch in train_progress:
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate predictions and update accuracy metrics
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

        # Update TQDM with the current batch loss and accuracy
        train_progress.set_postfix(loss=loss.item(), accuracy=correct_predictions / total_predictions)

    # Calculate epoch-level accuracy
    epoch_accuracy = correct_predictions / total_predictions

    # Print epoch-level loss and accuracy
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader):.4f}, Accuracy: {epoch_accuracy:.4f}")

Epoch 1: 100%|██████████| 313/313 [01:51<00:00,  2.80it/s, accuracy=0.596, loss=0.416]


Epoch 1, Loss: 0.8247, Accuracy: 0.5959


Epoch 2: 100%|██████████| 313/313 [01:52<00:00,  2.77it/s, accuracy=0.792, loss=0.519]


Epoch 2, Loss: 0.5197, Accuracy: 0.7918


Epoch 3: 100%|██████████| 313/313 [01:52<00:00,  2.77it/s, accuracy=0.851, loss=0.244]


...

Epoch 8, Loss: 0.0980, Accuracy: 0.9657


Epoch 9: 100%|██████████| 313/313 [01:52<00:00,  2.77it/s, accuracy=0.973, loss=0.0219] 


Epoch 9, Loss: 0.0828, Accuracy: 0.9726


Epoch 10: 100%|██████████| 313/313 [01:52<00:00,  2.77it/s, accuracy=0.976, loss=0.0718] 

Epoch 10, Loss: 0.0707, Accuracy: 0.9762


In [21]:
from sklearn.metrics import accuracy_score, classification_report


model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)

        outputs = model(**inputs)
        logits = outputs.logits

        # Get predictions (the index of the highest logit)
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(all_labels, all_preds, target_names=["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"]))

Validation Accuracy: 0.5806
Classification Report:
                 precision    recall  f1-score   support

       SUPPORTS       0.67      0.43      0.52       333
        REFUTES       0.73      0.57      0.64       333
NOT ENOUGH INFO       0.47      0.74      0.58       333

       accuracy                           0.58       999
      macro avg       0.62      0.58      0.58       999
   weighted avg       0.62      0.58      0.58       999



In [22]:
# Function to predict the class of a particular claim
def predict_claim_class(claim):
    # Preprocess the claim (same way as during training)
    inputs = tokenizer(claim, padding="max_length", truncation=True, max_length=128, return_tensors="pt").to(device)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (index of max logit)
    predicted_class_id = torch.argmax(logits, dim=1).item()

    # Convert class ID to label
    predicted_label = id2label[predicted_class_id]

    return predicted_label


In [23]:
# Example usage:
claim = "The Atlantic Ocean is the largest ocean on Earth."
predicted_label = predict_claim_class(claim)
print(f"The claim '{claim}' belongs to the class: {predicted_label}")

The claim 'The Atlantic Ocean is the largest ocean on Earth.' belongs to the class: SUPPORTS
